# **Smart-UJ UAQUE**
Este Notebook hace parte del servicio inteligente "perfil grupal" y tiene como objetivo la generación de clusters de usuarios a partir de el alquiler de ítems de la biblioteca Alfonso Borrero Cabal. Previamente se estrajeron los temas de cada ítem y en este notebook se generarán matrices dispersas que serán consumidad por el algoritmo de k-means para generar los clusters que posteriormente serán consumidos por los diferentes algoritmo de recomendación.

**Entrada**: Tabla_join.json

**Integrantes:**</br>
Johan Ortegón </br>
Juan Angarita </br>
Jorge Salgado </br>
Daniel Tibaquira </br></br>
**Directora de Proyecto de Grado:** </br>
Viky Arias </br>

# Importación de datos

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

Estas son las librerias que vamos a usar para hacer el join de ambas tablas y poder hacer las diferentes agrupaciones de estudiantes. 

* pandas(pd): Permite almacenar los datos en dataframes(matrices) y operación entre las celdas.
* KMeans: librería de sklearn para realizar el algoritmo de agrupamiento Kmeans.
* StandartScaler: Permite estandarizar los datos en una escala predeterminada.
* silhoute_score: permite sacar la métrica de puntaje de silhoute para los algoritmos de clustering.
* Normalizer: Permite normalizar los datos
* PCA: principal component analisis permite la reducción de dimensionalidad de nuestros dataset, en busca de que los algoritmos se ejecuten a mayor velocidad.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

Importamos el archivo. Este proviene del join entre las tablas de material bibliográfico de la biblioteca y la información de prestamos de la biblioteca. 

In [5]:
#df = pd.read_json('https://www.dropbox.com/s/chk4h90xgm48adm/joinTablas.json?dl=1')
df = pd.read_json('https://www.dropbox.com/s/c80i2zi3ba4a1lv/joinTablas-FEEDBACK.json?dl=1')

In [6]:
df.head(3)

,level_0,index,RowID,Fecha,IDItem,NumeroUbicacion,Dewey,Ubicacion,Llave,Programa,...,Calificación,Signatura,TipoItem,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,Temas,Union
0,0,0,Row0,1.547050e+12,80000005327627,720.98 A71S,720,COL-GRAL-2,866245,CARRERA DE ARQUITECTURA,...,NaN,720,LIBRO,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,2012.0,720.98,Arquitectura latinoamericana,"[arquitectur, histori, amer, latin]",arquitectur histori amer latin
1,1,1,Row1,1.547050e+12,80000001191496,720.9861 A71,720,COL-GRAL-2,309945,CARRERA DE ARQUITECTURA,...,NaN,720,LIBRO,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,1993.0,720.9861,Arquitectura colombiana,"[arquitectur, colonial, histori, colombi]",arquitectur colonial histori colombi
2,2,2,Row2,1.547139e+12,80000004979759,540 CH15Q 2010,540,COL-GRAL-3,822727,CARRERA DE ARQUITECTURA,...,NaN,540,LIBRO,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,2010.0,540,Química,[quimic],quimic


Anotamos la estructura del archivo. 

In [7]:
print("Numero de filas:", df.shape[0])
print("Numero de columnas:", df.shape[1])

Numero de filas: 489575
Numero de columnas: 23


El clustering para este experimento se realizará sobre el atributo "Signatura" el cual representa el dewey hasta la unidad (dando aproximadamente 900 deweys diferentes). Recordar la diferencia entre la columna "Dewey" y "Signatura". La primera es la que originalmente contiene el dataset de prestamos y solo venía con valores hasta la decena, mientras que "Sginatura" originalmente era una valor que señalaba el dewey especifico del libro y su ubicación al interior de la biblioteca. Se realizo una transformación para obtener únicamente el valor númerico de este valor y así obtener el dewey hasta la unidad(más especifico).

In [8]:
eliminar = df.loc[df["Signatura"].isnull()]
print("Numero de datos sin Dewey(Singatura):" , eliminar.shape[0])

Numero de datos sin Dewey(Singatura): 0


In [9]:
print("Valores de signatura unicos: " , len(df["Signatura"].unique()))
print("Valores de Dewey(decena) unicos: " , len(df["Dewey"].unique()))

Valores de signatura unicos:  756
Valores de Dewey(decena) unicos:  100


A partir del análisis con expertos en negocio, se ha entendido que es importante tomar en cuenta el cambio de gustos de los usuarios a través del tiempo y dar menos peso a prestamos que se realizaron en el pasado a prestamos más recientes. Para esto se creará una nueva columna denominada "Peso" la cual determinará el peso quue tiene dicho prestamo.
La disminución del peso será exponencial según la diferencia con el año actual y se calculará con la siguiente formula:

$Peso = \frac{1}{2^{(2021-20xx)}}$


In [10]:
def pesos(x):
    peso = 1/2**(2021-x.Year) 
    if x["Calificación"] == -1:
        peso = peso*-1
    return peso

In [11]:
df["Peso"] = df.apply(lambda row: pesos(row) , axis=1 )

In [12]:
df.loc[df.Peso < 0]

,level_0,index,RowID,Fecha,IDItem,NumeroUbicacion,Dewey,Ubicacion,Llave,Programa,...,Signatura,TipoItem,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,Temas,Union,Peso
489737,489737,0,None,NaN,80000004921734,None,None,None,None,None,...,780,LIBRO,None,"Western music and its others difference, repre...",2000.0,780.07,Música y sociedad,"[music, aspect, social]",music aspect social,-1.0
489738,489738,1,None,NaN,80000003200204,None,None,None,None,None,...,808,LIBRO,"Contursi, María Eugenia",La narración usos y teorías María Eugenia Cont...,2000.0,808,Retórica antigua,"[narracion, retor]",narracion retor,-1.0
489739,489739,2,None,NaN,80000004033284,None,None,None,None,None,...,711,LIBRO,"Saldarriaga Roa, Alberto 1941-","Bogotá siglo XX urbanismo, arquitectura y vida...",2006.0,None,None,"[urban, bogot, colombi]",urban bogot colombi,-1.0
489742,489742,5,None,NaN,80000005573117,None,None,None,None,None,...,371,LIBRO,"Pèz Martínez, Ruth Milena",Lo mítico en la formación de los niños una pro...,2017.0,371.102,Prácticas de la enseñanza,"[ensen, reflex]",ensen reflex,-1.0
489747,489747,10,None,NaN,80000003390692,None,None,None,None,None,...,752,LIBRO,"Küppers, Harald 1928-",Fundamentos de la teoría de los colores Harald...,1995.0,752,Color,[color],color,-1.0
489750,489750,13,None,NaN,80000001448045,None,None,None,None,None,...,330,LIBRO,"Tirado Mejía, Álvaro 1940-",Introducción a la historia económica de Colomb...,1983.0,None,None,"[economi, histori, colombi]",economi histori colombi,-1.0
489751,489751,14,None,NaN,80000004453556,None,None,None,None,None,...,301,LIBRO,"Latour, Bruno",Reensamblar lo social una introducción a la te...,2008.0,301.01,Teoría de la acción,"[filosofi, sociologi]",filosofi sociologi,-1.0
489752,489752,15,None,NaN,80000003592867,None,None,None,None,None,...,823,LITERATURA,"Christie, Agatha 1890-1976",Témoin muet (Dumb witness) Agatha Christie ; ...,1950.0,823,Novela tanzana (Inglés),"[novel, ingles]",novel ingles,-1.0
489753,489753,16,None,NaN,80000005575202,None,None,None,None,None,...,658,RESERVA,"McDaniel, Carl Daley",Investigación de mercados Carl McDaniel y Roge...,2016.0,658.83,Investigación de mercados,"[investigacion, merc]",investigacion merc,-1.0
489757,489757,20,None,NaN,80000002477654,None,None,None,None,None,...,720,LIBRO,"Norberg-Schulz, Christian",Arquitectura occidental La arquitectura como h...,1983.0,720.9,Arquitectura|xHistoria,"[arquitectur, histori]",arquitectur histori,-1.0


Se creará una nueva tabla donde únicamente se van a tener en cuenta los deweys que le gustan a cada usuario así como los atributos para posteriormente realizar el clustering. 

In [13]:
def positivo(x):
    if x<0:
        return 0
    else:
        return x

In [ ]:
pesos_usuarios = pd.DataFrame(columns= df["Signatura"].unique(),dtype=float)
tam = len(df["Signatura"].unique())
pesos_usuarios.columns = pesos_usuarios.columns.astype(str)
display(pesos_usuarios)

ids = df["IDUsuario"].unique()
i = 0
for usuario in ids:
  pesos_usuarios.loc[i] = np.zeros((tam,), dtype=int)
  #Para cada usuario traer sus Deweys y sus pesas asociados
  prestamos = df.loc[df["IDUsuario"]==usuario]
  deweys = prestamos[["Signatura","Peso"]]
  result_user = deweys.groupby("Signatura")["Peso"].sum().reset_index(name="Peso")
  result_user.Peso = result_user.Peso.apply(lambda x: positivo(x))
  # Recorrer los Deweys y Pesos para plasmarlos en una matriz
  #display(result_user)
  for index, row in result_user.iterrows():
    d = row["Signatura"]
    p=row['Peso']
    pesos_usuarios[d][i] = p
  i = i+1

,720,540,512,712,741,814,823,330,863,696,...,267,114,034,673,447,600,653,263,091,586


La matriz generada es una matriz dispersa donde todos los valores son ceros excepto donde el usuario haya realizado un prestamo.

In [ ]:
pesos_usuarios

# Configuración Modelos
A partir del archivo "Comparación modelos" se puede concluir que el modelo que ayuda en mayor medida a agrupar los datos, minimizando el número de clusters con tamaño menor a 20 y la desviación estandar es kmeans con los siguiente parámetros:
* Filas normalizadas = true
* Columnas normalizadas = true
* PCA = 0.95
* Número de clusters = 150
* Número de iteraciones = 500

In [ ]:
resumen = pd.DataFrame(columns = ["Tecnica","PCA", "Norm_filas", "Norm_col", "Precision", "Desviacion_estandar", "Media", "Coeficiente_variacion", "Clusters","Grupo_mas_grande", "Grupos<10", "Grupos<20"])


In [ ]:
def std_dev_cluster(data, labs):
    sizes_labels = []
    for label in np.unique(labs):
        sizes_labels.append(len(data.loc[data["cluster"] == label]))
    
    return np.std(sizes_labels)

In [ ]:
def mean_cluster(data, labs):
    sizes_labels = []
    for label in np.unique(labs):
        sizes_labels.append(len(data.loc[data["cluster"] == label]))
    
    return np.mean(sizes_labels)

In [ ]:
#
PCA = 0.95
num_clusters = 160
num_iter = 1000

# Algoritmo kmeans

In [ ]:
final = pesos_usuarios

In [ ]:
##Exportamos los datos del modelo que nos dio mejores resultados
#Normalización
#2.5 min por iteración
sumatoria = final.sum(axis=1)
pesos_norm = final.div(sumatoria, axis=0).fillna(0)

scaler = MinMaxScaler()
pesos_norm = scaler.fit_transform(pesos_norm.values)

  #Kmeans
k_means = KMeans(init = "k-means++", n_clusters = num_clusters, n_init = num_iter)
k_means.fit_predict(pesos_norm)
labels = k_means.labels_

In [ ]:
#Desempeño
#3 min
pres = silhouette_score(pesos_norm, labels)
print("silhouette_score: ", pres)

In [ ]:
grouped = df.groupby("IDUsuario")["Peso"].sum().reset_index(name="Peso")
grouped

In [ ]:
#copiamos el dataframe y agregamos los clusters para gráficar la distribución
df2 = final.copy()
df2["cluster"] = labels
df2.groupby("cluster")["cluster"].count()
display(df2["cluster"].value_counts().plot(kind='bar'))

#pegamos los cluster a la tabla que tiene los usuarios y los temas
grouped["cluster"] = labels


#Cálculo de métricas
media = mean_cluster(df2,labels)
desviacion = std_dev_cluster(df2,labels)
data = df2.groupby('cluster')['cluster'].count()
peque10 = len(data[data<=10])
peque20 = len(data[data<=20])
masGrande = data.max()
#Agregamos los datos a nuestra tabla de resumen para comparar
resumen = resumen.append({'Tecnica':"K-MEANS",
                              'PCA':0.95,
                              'Norm_filas':True,
                              'Norm_col':True,
                              'Precision':pres,
                              'Desviacion_estandar':desviacion,
                              'Media':media,
                              'Coeficiente_variacion':desviacion/media, 
                              'Clusters': num_clusters,
                              "Grupos<20":peque20,
                              "Grupos<10":peque10 , 
                              "Grupo_mas_grande":masGrande}, ignore_index=True)

In [ ]:
grouped.head(3)

In [ ]:
resumen

Podemos analizar los temas más importantes que tiene cada cluster. 

In [ ]:
cluster = df2.loc[df2.cluster == 89]
aux=cluster.astype(bool).sum(axis=0)
df3 = aux[aux > 0]
df3.sort_values(ascending=False).head(20)

In [ ]:
grouped.head(3)

Pegamos la información del cluster a cada uno de los prestamos de los usuarios.(temas_usuarios_cluster)

In [ ]:
material_data = pd.DataFrame(data=df)
cluster_data = pd.DataFrame(data=grouped)
temas_usuarios_cluster = pd.merge(material_data, cluster_data, left_on='IDUsuario', right_on='IDUsuario', how='left')
temas_usuarios_cluster.head(3)

Crearemos un tercer dataframe el cual contega únicamente la información de cada cluster(sus centroides). 

In [ ]:
centroids = pd.DataFgrame(data = k_means.cluster_centers_)

In [ ]:
centroids.columns = df["Signatura"].unique()
centroids

In [ ]:
centroids[centroids<1e-5] = 0

Finalmente exportamos las tablas para ser consumidas por los sistemas de recomedación junto al dashboard.
* Grouped_temas = usuarios + cluster
* temas_usuarios_cluster = joinTablas + cluster
* Centroids_deweys = clusters + centroides

In [ ]:
grouped.to_json('../data/GruposDeweyUnidad/userCluster.json')

In [ ]:
temas_usuarios_cluster.to_json('../data/GruposDeweyUnidad/joinCluster.json')

In [ ]:
centroids.to_json('../data/GruposDeweyUnidad/centroids.json')